In [1]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

2025-12-04 13:52:51.529143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764856371.971963      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764856372.126995      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# DATA LOADING & INTEGRITY CHECK
base_dir = os.path.join('..', 'input', 'skin-cancer-mnist-ham10000')
image_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob.glob(os.path.join(base_dir, '*', '*.jpg'))}
skin_df = pd.read_csv(os.path.join(base_dir, 'HAM10000_metadata.csv'))
skin_df['path'] = skin_df['image_id'].map(image_path_dict.get)

In [6]:
def set_target(dx):
    return 'cancer' if dx in ['mel', 'bcc', 'akiec'] else 'safe'
skin_df['target'] = skin_df['dx'].apply(set_target)

In [7]:
train_df_raw, val_df = train_test_split(skin_df, test_size=0.2, random_state=42, stratify=skin_df['target'])

In [8]:
# CLASS BALANCING (TRAIN SET ONLY)
df_safe = train_df_raw[train_df_raw['target'] == 'safe']
df_cancer = train_df_raw[train_df_raw['target'] == 'cancer']

# Upsample minority class to match majority
df_cancer_upsampled = resample(df_cancer, replace=True, n_samples=len(df_safe), random_state=42)
train_df = pd.concat([df_safe, df_cancer_upsampled]) # Balanced Training Set

print(f"Train Set (Balanced): {len(train_df)}")
print(f"Validation Set (Natural/Imbalanced): {len(val_df)}")

Train Set (Balanced): 12898
Validation Set (Natural/Imbalanced): 2003


In [9]:
# IMAGE GENERATORS (Standardized)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [10]:
# Augmentation: Enough to learn invariance, but not distorting the lesion structure
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df, x_col='path', y_col='target',
    target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df, x_col='path', y_col='target',
    target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)

Found 12898 validated image filenames belonging to 2 classes.
Found 2003 validated image filenames belonging to 2 classes.


In [11]:
# MODEL ARCHITECTURE (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Start Frozen
base_model.trainable = False 

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(), # Added for training stability
    Dropout(0.4),         # High dropout to reduce overfitting
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Metrics: AUC is often better than Accuracy for medical data
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy', 'Recall', 'AUC'])

I0000 00:00:1764856637.666941      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764856637.667752      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# HEAD TRAINING
history1 = model.fit(
    train_generator, 
    epochs=5, 
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


I0000 00:00:1764856671.893050     121 service.cc:148] XLA service 0x7804bc111c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764856671.894580     121 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764856671.894604     121 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764856673.000622     121 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/404 ━━━━━━━━━━━━━━━━━━━━ 1:27:58 13s/step - AUC: 0.2686 - Recall: 0.1333 - accuracy: 0.3438 - loss: 1.1391

I0000 00:00:1764856678.633950     121 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


404/404 ━━━━━━━━━━━━━━━━━━━━ 357s 854ms/step - AUC: 0.7911 - Recall: 0.7047 - accuracy: 0.7222 - loss: 0.6229 - val_AUC: 0.8531 - val_Recall: 0.7115 - val_accuracy: 0.7334 - val_loss: 0.5160
Epoch 2/5
404/404 ━━━━━━━━━━━━━━━━━━━━ 218s 540ms/step - AUC: 0.8600 - Recall: 0.7544 - accuracy: 0.7783 - loss: 0.4662 - val_AUC: 0.8680 - val_Recall: 0.7029 - val_accuracy: 0.7314 - val_loss: 0.5235
Epoch 3/5
404/404 ━━━━━━━━━━━━━━━━━━━━ 218s 541ms/step - AUC: 0.8777 - Recall: 0.7604 - accuracy: 0.7941 - loss: 0.4340 - val_AUC: 0.8693 - val_Recall: 0.7537 - val_accuracy: 0.7654 - val_loss: 0.4608
Epoch 4/5
404/404 ━━━━━━━━━━━━━━━━━━━━ 214s 529ms/step - AUC: 0.8935 - Recall: 0.7896 - accuracy: 0.8114 - loss: 0.4061 - val_AUC: 0.8791 - val_Recall: 0.7438 - val_accuracy: 0.7624 - val_loss: 0.4563
Epoch 5/5
404/404 ━━━━━━━━━━━━━━━━━━━━ 216s 535ms/step - AUC: 0.8965 - Recall: 0.7842 - accuracy: 0.8160 - loss: 0.3998 - val_AUC: 0.8758 - val_Recall: 0.7543 - val_accuracy: 0.7723 - val_loss: 0.4610


In [13]:
# FINE TUNING
base_model.trainable = True
# Freeze early layers (structural features), Unfreeze late layers (texture features)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Compile with very low LR to nudge weights gently
model.compile(optimizer=Adam(learning_rate=1e-5), 
              loss='binary_crossentropy', 
              metrics=['accuracy', 'Recall', 'AUC'])

callbacks = [
    ModelCheckpoint('best_skin_model.h5', save_best_only=True, monitor='val_auc', mode='max', verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1)
]

In [14]:
history2 = model.fit(
    train_generator, 
    epochs=25, # Max epochs, EarlyStopping will stop it earlier
    validation_data=val_generator, 
    callbacks=callbacks
)

Epoch 1/25
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 530ms/step - AUC: 0.8113 - Recall: 0.6868 - accuracy: 0.7339 - loss: 0.5326

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_auc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


404/404 ━━━━━━━━━━━━━━━━━━━━ 256s 582ms/step - AUC: 0.8114 - Recall: 0.6868 - accuracy: 0.7340 - loss: 0.5325 - val_AUC: 0.8568 - val_Recall: 0.8610 - val_accuracy: 0.8198 - val_loss: 0.4637 - learning_rate: 1.0000e-05
Epoch 2/25
404/404 ━━━━━━━━━━━━━━━━━━━━ 215s 533ms/step - AUC: 0.8674 - Recall: 0.7397 - accuracy: 0.7819 - loss: 0.4536 - val_AUC: 0.8712 - val_Recall: 0.8635 - val_accuracy: 0.8263 - val_loss: 0.3984 - learning_rate: 1.0000e-05
Epoch 3/25
404/404 ━━━━━━━━━━━━━━━━━━━━ 216s 534ms/step - AUC: 0.8919 - Recall: 0.7686 - accuracy: 0.8102 - loss: 0.4083 - val_AUC: 0.8755 - val_Recall: 0.8071 - val_accuracy: 0.7953 - val_loss: 0.4291 - learning_rate: 1.0000e-05
Epoch 4/25
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - AUC: 0.8929 - Recall: 0.7734 - accuracy: 0.8150 - loss: 0.4075
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
404/404 ━━━━━━━━━━━━━━━━━━━━ 216s 535ms/step - AUC: 0.8929 - Recall: 0.7734 - accuracy: 0.8150 - loss: 0.4074 - val_AUC: 0.883

In [15]:
from IPython.display import FileLink
FileLink(r'best_skin_model.h5')

/kaggle/working/best_skin_model.h5

In [16]:
model.save('backup_model.h5')
print("Model manually saved as 'backup_model.h5'")

import os
if os.path.exists('backup_model.h5'):
    print("✅ File Found! Generating link...")

    from IPython.display import FileLink
    display(FileLink(r'backup_model.h5'))
else:
    print("❌ File still not found. Check Output directory.")

Model manually saved as 'backup_model.h5'
✅ File Found! Generating link...


/kaggle/working/backup_model.h5